<a href="https://colab.research.google.com/github/siva4iov/Amazon_Employee_Access_Challenge/blob/main/Amazon_com_Employee_Access_Challenge_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание используемых алгоритмов

## train_test_split
Используем train_test_split для создания валидационной выборки, важно использовать stratify, так как данные с очень нарушенным балансом, нужно чтобы одинаковое кол-во экземпляров класса попало и в тренировочную и валидационную

# GaussianNB
Для начала возьмем простые модели, позволяющие определить на что вообще можно расчитывать, гауссова наивная модель мне кажется одна из самых простейших моделей(не считая линейных) которые мы можем попробовать

# GridSearchCV
Для нахождения оптимальных гиперпараметров, будем использовать GridSearch, для отбора fold'ов мы использовали **StratifiedShuffleSplit**, причиной этого было, что нам нужно равное распределение меньшего класса по fold'ам, а также, предпологалось использовать для **GridSearch** не всю выборку, этот алгоритм это позволяет. Но в конечном итоге мы решили использовать байесовскую модель, а она довольная легкая и можно было брать всю выборку.

Параметрами **GridSearch** было **var_smoothing**, точнее доля дисперсии признаков.

### GaussianNB score
Точность на вал выборке была 0.94, но при этом точность на данных, содержащих метку меньшего класса, точность 0.
То есть модель сильно переобучилась и в итоге практически просто выдает константу 1

### Варианты решения
Из-за того, что доля меньшего класса около 5%, следует использовать другую стратегию.

Мы предположили нескольлко вариантов решения:

    - Использовать другую модель, которая лучше подходит для таких данных, например дерево решений
    - Использовать undersamling или oversampling
    - Использовать для отбора признаков другие метрики, которые лучше подходят для несбалансированных данных

# GradientBoostingClassifier

Мы решили использовать сразу бустинг, вместо просто дерева, но начали с модели от sklearn. У нее намного меньше полезных фич, в отличие от XGBoost или CatBoost.

Параметрами gridsearch мы использовали только learning_rate и n_estimators, так как это самые значимые гиперпараметры.

Также, тут для отбора fold'ов уже используем **StratifiedKFold**, так как это не позволит данным в fold'ах пересекаться, а нам нужны все наши значения, особенно с меткой 0. Плюс ко всему, мы используем 5 фолдов, опять же чтобы оценка была сбалансированнее и мы использовали больше данных.

Мы опробовали несколько метрик, таких как *recall, precision, f1*, но в итоге остановились на **ROC_AUC**. К тому же только тут мы заметили, что для оценки на kaggle используется эта же метрика.

Это дало небольшой прирост в точности на данных с метрикой 0, но этого все равно недостаточно.


# Oversampling

Мы решили не обрезать данные, а синтезировать дополнительные.

Метод был выбран **SMOTE** из модуля **imblearn**, со значением k_neighbors = 4, мы попробовали брать значение и поменьше, но это не дало прироста в качестве.

На всякий случай, мы дополнительно прогнали **GridSearch** на пересэмплированных данных, лкчшие гиперпараметры все равно остались прежними

Точность на значениях с меткой 0, значительно выросло. Мы решили сделать submission на kaggle, результат оказался уже довольно неплохой, **0.80229**. По большей части это вызванно тем, что после того как мы осознали что метрика для submission *roc_auc*,
мы стали использовать значения predict_proba

# Catboost 

На этом этапе мы решили перейти к более продвинутым моделям, таким как **catboost**.

Cперва мы решили найти лучший learning_rate посредством gridsearch, модель взяли с 500 итерациями для ускорения процесса, как функцию потерь мы взяли КроссЭнтропию, это хорошая практика для данных с таким распределением.

Лучшим значением learning_rate стало 0.1, можно было бы дополнительно пройти gridsearch и попробовать значения {0.05, 0.2},
но мы посчитали это излишним, так как мы не гонимся за улучшением сотых долей результата.

Как модель для предсказаний мы взяли модель с 1000 итераций, потому что с большим значением пришлось бы довольно ждать.

Эта модель выдала очень хорошую оценку на валидационной выборке  -  **0.95172**

# Результат

Финальная оценка на kaggle равна -  **0.87788**
При этом лучшая оценка лидерборда -  **0.92964**
Результат показался нам довольно неплохим и мы остались удовлетворены таким результатом.